# Data science project for pricing cars 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels
import statsmodels.api as sm
import seaborn as sns

#Set the style options
%matplotlib inline
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
plt.style.use('seaborn')

In [ ]:
#import the file
df_cars = pd.read_csv('data/city_30.csv', index_col = 0, header = 0)

#Drop picture for the sake of file size
df_cars = df_cars.drop('main_picture_url', axis = 1)
df_cars = df_cars.drop('description', axis = 1)

In [ ]:
df_cars.info()

# view info

## After importing and viewing the file, begin to explore the data. take note of Dtypes, non-null counts, and try to figure out what features are worth keeping or removing.

In [ ]:
df_cars['fleet'].describe()

# describe fleet, note null values

In [ ]:
df_cars.head()

# view head

In [ ]:
df_cars['transmission'].value_counts()

# value counts for transmission

In [ ]:
df_cars['frame_damaged'].value_counts()

# value counts for frame damaged

In [ ]:
df_copy = df_cars

# make a copy

In [ ]:
df_cars = df_cars.drop(['vehicle_damage_category', 'combine_fuel_economy', 'is_certified'], axis = 1)

#drop the columns with no values

In [ ]:
df_cars.info()

# view info

In [ ]:
def new_car(c):
    if c['is_new']:
        return 1
    else:
        return 0
    
df_cars['new'] = df_cars.apply(new_car,axis = 1)

# engineer features and create new

In [ ]:
df_cars['new'].describe()

# describe new

In [ ]:
df_cars.head()

# view head

In [ ]:
df_cars['is_cpo'].value_counts()

# value counts for certified pre owned

In [ ]:
from sklearn.preprocessing import LabelEncoder
lc = LabelEncoder()

# import library

In [ ]:
df_cars['is_cpo'] = lc.fit_transform(df_cars['is_cpo'])
df_cars['is_cpo'].value_counts()

# encode values for is_cpo

In [ ]:
df_cars.info()

# view info

In [ ]:
df_cars['dealer_zip'].value_counts()

# value counts for dealer_zip

In [ ]:
df_cars['city'].value_counts()

# value_counts for city

In [ ]:
df_reg = pd.DataFrame()

# beginning the dataframe for the regression model. We will add to the dataframe as we go.

In [ ]:
df_reg['year'] = df_cars['year']

# add year to regression model dataframe

In [ ]:
df_reg.info()

#view info

In [ ]:
df_reg['is_new'] = df_cars['new']

# add is_new to regression dataframe

In [ ]:
df_reg['is_cpo'] = df_cars['is_cpo']

# add is_cpo to new dataframe

In [ ]:
df_reg.info()

# view new dataframe

In [ ]:
df_cars['city_fuel_economy'].value_counts()

# value_counts for city_fuel_economy

In [ ]:
df_cars['city_fuel_economy'].describe()

# describe city_fuel_economy

In [ ]:
df_cars['city_fuel_economy'].isna()

# search for missing values for city_fuel_economy

In [ ]:
df_cars[df_cars['city_fuel_economy'].isna()].sample(20)

# view sample of na for city_fuel_economy

In [ ]:
df_cars['city_fuel_economy'] = df_cars['city_fuel_economy'].fillna(22)

# fill null values with median

In [ ]:
df_cars['city_fuel_economy'].value_counts()

# value_counts for city_fuel_economy

In [ ]:
df_reg['city_fuel_economy'] = df_cars['city_fuel_economy']

# add city_fuel_economy to regression dataframe

In [ ]:
df_cars['highway_fuel_economy'].describe()

# describe highway fule economy

In [ ]:
df_cars['highway_fuel_economy'] = df_cars['highway_fuel_economy'].fillna(29)

# fill with median value from above

In [ ]:
df_reg['highway_fuel_economy'] = df_cars['highway_fuel_economy']

# add highway_fuel_economy to new dataframe

In [ ]:
df_cars.info()

# view info for cars

In [ ]:
def accident(c):
    if c['has_accidents']:
        return 1
    else:
        return 0
    
df_cars['accidents'] = df_cars.apply(accident,axis = 1)

# engineer features and create accidents

In [ ]:
df_cars['accidents'].value_counts()

# view value_counts

In [ ]:
df_reg['has_accident'] = df_cars['accidents']

# add accidents to regression dataframe

In [ ]:
df_reg.info()

# view info

In [ ]:
df_cars['owner_count'].value_counts()

# view value_counts for number of owners

In [ ]:
df_cars['owner_count'] = df_cars['owner_count'].fillna(0)

# fill missing values with 0 assuming that these cars are new

In [ ]:
df_reg['owner_count'] = df_cars['owner_count']

# add owner_count to regression dataframe

In [ ]:
df_reg.info()

# view info for regression model dataframe

In [ ]:
df_cars['fuel_type'].value_counts()

# view value_counts for fuel type

In [ ]:
df_cars['fuel_type'] = df_cars['fuel_type'].fillna('Unknown')

# fill na with 'Unknown'

In [ ]:
df_cars['fuel_type'].value_counts()

# view value_counts again

In [ ]:
dummies_ft = pd.get_dummies(df_cars['fuel_type'], drop_first = False, prefix = 'ft', dtype = 'int')

# dummy variables for fuel type

In [ ]:
dummies_ft.sample(15, random_state = 42)

# view sample of dummy variables 

In [ ]:
dummies_ft = dummies_ft.drop('ft_Gasoline', axis = 1)

# drop gasoline

In [ ]:
df_reg = pd.concat([df_reg, dummies_ft], axis = 1)

# add dummy variables to regression dataframe

In [ ]:
df_reg.info()

# view info

In [ ]:
df_cars['wheel_system'].value_counts()

# value_counts for wheel system

In [ ]:
df_cars['wheel_system'] = df_cars['wheel_system'].fillna('Unknown')

# fill na with unknown

In [ ]:
dummies_ws = pd.get_dummies(df_cars['wheel_system'], drop_first = False, prefix = 'ws', dtype = 'int')

#dummy variables for wheel_system

In [ ]:
dummies_ws.sample(15, random_state = 42)

# view sample of dummy variables 

In [ ]:
df_reg = pd.concat([df_reg, dummies_ws], axis = 1)

# add dummy variables to df_reg

In [ ]:
df_reg = df_reg.drop('ws_FWD', axis = 1)

# drop ws_FWD

In [ ]:
df_reg.info()

# view info

In [ ]:
df_cars[df_cars['mileage'] == 0]

# set mileage to 0

In [ ]:
df_cars['mileage'] = df_cars['mileage'].fillna(0)

# assume that cars with na for mileage are new

In [ ]:
df_cars.info()

# view info

In [ ]:
df_reg['mileage'] = df_cars['mileage']

# add mileage to df_reg

In [ ]:
df_reg.info()

# view info

In [ ]:
df_reg['days_on_market'] = df_cars['daysonmarket']

# add days_on_market to df_reg

In [ ]:
df_reg['price'] = df_cars['price']

# add price to df_reg

## After initial cleaning, begin to make models.

In [ ]:
X = df_reg.drop('price', axis = 1)

y = df_reg['price']

X = sm.add_constant(X)

reg = sm.OLS(y, X).fit()
pred = reg.predict(X)
resid = reg.resid

reg.summary()

# multiple regression 1

## Clearly this is not a good model. The R^2 is not good and there are non significant P values.

In [ ]:
#Import the library for VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

#Create the VIF for the features dataframe
vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(X.values, i) for i in range(1, X.shape[1])]

#Add a column for labels
vif['Features'] = X.columns[1:]

round(vif, 1).sort_values(by = 'VIF Factor', ascending = False)

In [ ]:
df_cars.sort_values('make_name')

# sort by make_name

In [ ]:
df_cars['make_name'].value_counts()

# value counts for make_name

In [ ]:
pd.set_option('display.max_rows', 100)

# setting pandas to view more rows

In [ ]:
df_cars['make_name'].value_counts()

# value counts for make_name

In [ ]:
df_lowsales = pd.DataFrame()

#Dataframe to hold low sales - dropping makes with counts < 1000 due to wide variety in prices and makes in small counts

In [ ]:
counts = df_cars['make_name'].value_counts()

df_lowsales = df_cars[~df_cars['make_name'].isin(counts[counts > 1000].index)]

#Creating a series of value counts
# Uses the counts series to determine if the make_name is a category with lower than 1000 values
#df_lowsales hold values less than 1000

In [ ]:
df_lowsales.sample(75)

# view sample or low sales

In [ ]:
df_lowsales.shape

# view shape of low sales

In [ ]:
df_lowsales['make_name'].value_counts()

# value counts of make_name

In [ ]:
df_reg['make_name'] = df_cars['make_name']

# add make name to df_reg 

In [ ]:
cond = df_cars['make_name'].isin(df_lowsales['make_name'])

# create a series of rows that are in both

df_cars.drop(df_cars[cond].index, inplace = True)
df_reg.drop(df_reg[cond].index, inplace = True)

# Drops anything from that series in df_cars and df_reg 

In [ ]:
df_reg.info()

# view info 

In [ ]:
dummies_mn = pd.get_dummies(df_reg['make_name'], drop_first = False, prefix = 'mn', \
                            dtype = 'int')

# dummy variable make_name

In [ ]:
df_reg = pd.concat([df_reg, dummies_mn], axis = 1)

# add dummy variables to regression dataframe

In [ ]:
df_reg = df_reg.drop(['mn_Ford', 'make_name'], axis = 1)
df_reg.info()

# drop columns and view info

In [ ]:
df_cars['model_name'].value_counts()

# value counts for model name

In [ ]:
df_c_pivot = df_cars.pivot_table(index='make_name', columns = 'model_name', values = 'price')

#Create a pivot table for 'Model Name' by 'Make Name'

df_c_pivot

#Display the pivot table

In [ ]:
df_cars['body_type'].value_counts()

# value counts for body type

In [ ]:
dummies_bt = pd.get_dummies(df_cars['body_type'], drop_first = False, prefix = 'bt', \
                            dtype = 'int')
df_reg = pd.concat([df_reg, dummies_bt], axis = 1)
df_reg.info()

#create dummy variables for body type and add them into df_reg

In [ ]:
df_reg = df_reg.drop('bt_SUV / Crossover', axis = 1)

#drop SUV/Crossover because it has the largest number of counts

In [ ]:
df_reg.info()  

# view info

In [ ]:
df_cars['city'].value_counts()

# value counts for city

In [ ]:
dummies_ci = pd.get_dummies(df_cars['city'], drop_first = False, prefix = 'ci', \
                            dtype = 'int')
df_reg = pd.concat([df_reg, dummies_ci], axis = 1)
df_reg.info()

#Dummy variables for city

In [ ]:
df_reg = df_reg.drop('ci_Houston', axis = 1)

#drop Houston from df_reg

In [ ]:
df_reg.info()

# view the ever growing df_reg dataframe

In [ ]:
df_cars['engine_type'].value_counts()

# value counts for engine type

In [ ]:
df_cars[df_cars['maximum_seating'].isna()]

# view na for maximum seating

In [ ]:
df_cars['maximum_seating'].value_counts()

# value_counts for maximum_seating

In [ ]:
df_cars['maximum_seating'].describe()

# describe maximum_seating

In [ ]:
df_cars['maximum_seating'] = df_cars['maximum_seating'].fillna("5 seats")

# fill na values in maximum seating with 5 seats. Assuming since there are so many values for 5 seats, the missing values being added will not negatively impact the model

In [ ]:
df_cars['maximum_seating'].value_counts()

# value counts for maximum seating with na values filled

In [ ]:
df_cars[df_cars['maximum_seating'] == '--']

# viewing -- in maximum_seating

In [ ]:
df_cars = df_cars.replace({'--' : '5 seats'})

# replace the value -- with 5 seats for the same reason as above

In [ ]:
df_cars['maximum_seating'].value_counts()

# value_counts for maximum_seating with na and -- filled 

In [ ]:
dummies_ms = pd.get_dummies(df_cars['maximum_seating'], drop_first = False, prefix = 'ms', \
                            dtype = 'int')
df_reg = pd.concat([df_reg, dummies_ms], axis = 1)
df_reg.info()

# dummy variable maximum seats

In [ ]:
df_reg = df_reg.drop('ms_5 seats', axis = 1)

# drop 5 seats because it has the most

In [ ]:
df_reg.info()

# view info

In [ ]:
X = df_reg.drop('price', axis = 1)

y = df_reg['price']

X = sm.add_constant(X)

reg = sm.OLS(y, X).fit()
pred = reg.predict(X)
resid = reg.resid

reg.summary()

# second regression model, including more features

## The second regression model shows a significantly better value for R^2, which is a huge improvement thanks to the inclusion of more features. It still shows many features with p values that are not significant. Clean those after further improving the model.

In [ ]:
# Create a residual plot
plt.figure(figsize = (20, 16))
sns.jointplot(x = pred, y = resid, kind = 'reg', ratio = 5, height = 8, \
             line_kws = {'color' : 'black'})

#Add x and y labels
plt.xlabel('Predicted Sale Price', fontweight = 'bold', fontsize = 14, horizontalalignment = 'center')
plt.ylabel('Residual', fontweight = 'bold', fontsize = 14, horizontalalignment = 'center')

## The residual plot shows that WE WILL PAY ***YOU*** TO TAKE OUR CARS COME ON DOWN NOW AND GET YOURS!!

## The many predicted sales prices below zero are an issue that needs to be fixed as well as the oddly high residuals around the 50000 mark.

In [ ]:
df_cars.info()

# view info

In [ ]:
def leather_seats(c):
    ls = 'Leather Seats'
    list_options = str(c['major_options'])
    
    if list_options.find(ls) >= 0:
        return 1
    else:
        return 0

    
df_reg['leather_seats'] = df_cars.apply(leather_seats, axis = 1)

#Create a function to pull a feature out of the major_options category

In [ ]:
df_reg.head()

# view head

In [ ]:
def bluetooth(c):
    bt = 'Bluetooth'
    list_options = str(c['major_options'])
    
    if list_options.find(bt) >= 0:
        return 1
    else:
        return 0

    
df_reg['bluetooth'] = df_cars.apply(bluetooth, axis = 1)

#Create a function to pull a feature out of major_options

In [ ]:
df_reg.head()

# view head

In [ ]:
def backupcam(c):
    bc = 'Backup Camera'
    list_options = str(c['major_options'])
    
    if list_options.find(bc) >= 0:
        return 1
    else:
        return 0

    
df_reg['backup_camera'] = df_cars.apply(backupcam, axis = 1)

#Create a function to pull a feature out of major_options

In [ ]:
df_reg.head()

# view head

In [ ]:
def sunmoon(c):
    sm = 'Sunroof/Moonroof'
    list_options = str(c['major_options'])
    
    if list_options.find(sm) >= 0:
        return 1
    else:
        return 0

    
df_reg['Sunroof/Moonroof'] = df_cars.apply(sunmoon, axis = 1)
df_reg.head()

#Create a function to pull a feature out of major_options

In [ ]:
def phonecap(c):
    cp = 'CarPlay'
    aa = 'Android Auto'
    list_options = str(c['major_options'])
    
    if list_options.find(cp) >= 0 or list_options.find(aa) >=0:
        return 1
    else:
        return 0

    
df_reg['Phone Capability'] = df_cars.apply(phonecap, axis = 1)
df_reg.head()

#Create a function to pull a feature out of major_options

In [ ]:
X = df_reg.drop('price', axis = 1)

y = df_reg['price']

X = sm.add_constant(X)

reg = sm.OLS(y, X).fit()
pred = reg.predict(X)
resid = reg.resid

reg.summary()

# multiple regression number 3

In [ ]:
# Create a residual plot
plt.figure(figsize = (20, 16))
sns.jointplot(x = pred, y = resid, kind = 'reg', ratio = 5, height = 8, \
             line_kws = {'color' : 'black'})

#Add x and y labels
plt.xlabel('Predicted Sale Price', fontweight = 'bold', fontsize = 14, horizontalalignment = 'center')
plt.ylabel('Residual', fontweight = 'bold', fontsize = 14, horizontalalignment = 'center')

## The third multiple regression and residual plot does not show much of a difference from the second one, Indicating that pulling out features from major options did not have much of an effect.

In [ ]:
features = X.drop('const', axis = 1)

# Create a dataframe that includes the X dataset without the constant

corrMatrix = features.corr()

# Create a corrMatrix

df_corrMatrix = pd.DataFrame(corrMatrix)
round(df_corrMatrix, 3)

#Convert corrMatrix to a dataframe

In [ ]:
#Set the colormap for the heatmap
colormap = plt.cm.RdBu

#Set the size for the figure
plt.figure(figsize = (40, 40))

#Create the heatmap
sns.heatmap(df_corrMatrix, linewidths = 0.1, vmax = 1.0, vmin = -1.0, \
            square = True, cmap = colormap, linecolor = 'white', annot = False)

## The heatmap shows significant correlation in the top left corner that seems to make sense.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

#Import the library for VIF

vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(X.values, i) for i in range(1, X.shape[1])]

#Import the library for VIF

vif['Features'] = X.columns[1:]

round(vif, 1).sort_values(by = 'VIF Factor', ascending = False)

#Add a column for labels

## The VIF shows multicollinearity for the two fuel economy features.

In [ ]:
df_reg = df_reg[df_reg['year'] >= 2015]

# Drop the cars made before 2010

In [ ]:
df_reg.info()

# view info

In [ ]:
df_reg.head()

# view head

In [ ]:
df_reg['year'].value_counts().sum()

# view sum of value counts for year

In [ ]:
X = df_reg.drop('price', axis = 1)

y = df_reg['price']

X = sm.add_constant(X)

reg = sm.OLS(y, X).fit()
pred = reg.predict(X)
resid = reg.resid

reg.summary()

# fourth regression model

In [ ]:
# Create a residual plot
plt.figure(figsize = (20, 16))
sns.jointplot(x = pred, y = resid, kind = 'reg', ratio = 5, height = 8, \
             line_kws = {'color' : 'black'})

#Add x and y labels
plt.xlabel('Predicted Sale Price', fontweight = 'bold', fontsize = 14, horizontalalignment = 'center')
plt.ylabel('Residual', fontweight = 'bold', fontsize = 14, horizontalalignment = 'center')

## The fourth regression and residual plot dont show a significant difference in the R^2 value, but we finally stopped giving away money and cars.

In [ ]:
features = X.drop('const', axis = 1)

# Create a dataframe that includes the X dataset without the constant

corrMatrix = features.corr()

# Create a corrMatrix

df_corrMatrix = pd.DataFrame(corrMatrix)
round(df_corrMatrix, 3)

#Convert corrMatrix to a dataframe

In [ ]:
#Set the colormap for the heatmap
colormap = plt.cm.RdBu

#Set the size for the figure
plt.figure(figsize = (40, 40))

#Create the heatmap
sns.heatmap(df_corrMatrix, linewidths = 0.1, vmax = 1.0, vmin = -1.0, \
            square = True, cmap = colormap, linecolor = 'white', annot = False)

In [ ]:
#Import the library for VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

#Create the VIF for the features dataframe
vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(X.values, i) for i in range(1, X.shape[1])]

#Add a column for labels
vif['Features'] = X.columns[1:]

round(vif, 1).sort_values(by = 'VIF Factor', ascending = False)

In [ ]:
df_reg['comb_fuel_economy'] = (df_reg['highway_fuel_economy'] + df_reg['city_fuel_economy']) / 2
df_reg = df_reg.drop(['highway_fuel_economy', 'city_fuel_economy'], axis = 1)

#combine fuel econ

In [ ]:
df_reg.info()

# view info

In [ ]:
X = df_reg.drop('price', axis = 1)

y = df_reg['price']

X = sm.add_constant(X)

reg = sm.OLS(y, X).fit()
pred = reg.predict(X)
resid = reg.resid

reg.summary()

# fifth regression

In [ ]:
# Create a residual plot
plt.figure(figsize = (20, 16))
sns.jointplot(x = pred, y = resid, kind = 'reg', ratio = 5, height = 8, \
             line_kws = {'color' : 'black'})

#Add x and y labels
plt.xlabel('Predicted Sale Price', fontweight = 'bold', fontsize = 14, horizontalalignment = 'center')
plt.ylabel('Residual', fontweight = 'bold', fontsize = 14, horizontalalignment = 'center')

## Finally at the fifth regression and residual plot, after combining the fuel economies, progress is being made. 

In [ ]:
features = X.drop('const', axis = 1)

# Create a dataframe that includes the X dataset without the constant

corrMatrix = features.corr()

# Create a corrMatrix

df_corrMatrix = pd.DataFrame(corrMatrix)
round(df_corrMatrix, 3)

#Convert corrMatrix to a dataframe

In [ ]:
#Set the colormap for the heatmap
colormap = plt.cm.RdBu

#Set the size for the figure
plt.figure(figsize = (40, 40))

#Create the heatmap
sns.heatmap(df_corrMatrix, linewidths = 0.1, vmax = 1.0, vmin = -1.0, \
            square = True, cmap = colormap, linecolor = 'white', annot = False)

In [ ]:
#Import the library for VIF
from statsmodels.stats.outliers_influence import variance_inflation_factor

#Create the VIF for the features dataframe
vif = pd.DataFrame()
vif['VIF Factor'] = [variance_inflation_factor(X.values, i) for i in range(1, X.shape[1])]

#Add a column for labels
vif['Features'] = X.columns[1:]

round(vif, 1).sort_values(by = 'VIF Factor', ascending = False)

In [ ]:
X = X.drop(['bt_Hatchback', 'ci_Las Vegas', 'ci_Raleigh', 'ci_Scottsdale'], axis = 1)

y = df_reg['price']

X = sm.add_constant(X)

reg = sm.OLS(y, X).fit()
pred = reg.predict(X)
resid = reg.resid

reg.summary()

# fifth regression

In [ ]:
X = X.drop('ft_Compressed Natural Gas', axis = 1)

y = df_reg['price']

X = sm.add_constant(X)

reg = sm.OLS(y, X).fit()
pred = reg.predict(X)
resid = reg.resid

reg.summary()

# sixth regression

In [ ]:
X = X.drop('ci_Columbia', axis = 1)

y = df_reg['price']

X = sm.add_constant(X)

reg = sm.OLS(y, X).fit()
pred = reg.predict(X)
resid = reg.resid

reg.summary()

# seventh regression

In [ ]:
# Create a residual plot
plt.figure(figsize = (20, 16))
sns.jointplot(x = pred, y = resid, kind = 'reg', ratio = 5, height = 8, \
             line_kws = {'color' : 'black'})

#Add x and y labels
plt.xlabel('Predicted Sale Price', fontweight = 'bold', fontsize = 14, horizontalalignment = 'center')
plt.ylabel('Residual', fontweight = 'bold', fontsize = 14, horizontalalignment = 'center')

### After removing more features with a non significant p-value, the R^2 and odd residuals did not change. Overall the model works pretty well considering it is broad and not based on a specific brand or model of car. Considering the broad range of values, and the very large dataset, the end model is a pretty good indicator of the predicted sale price of a car.